In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

#### Create a list of documents

In [5]:
docs = [
"This is the first document.",
"This is the second document. But this is not the first document.",
"This is the third document. It is neither the first or the second document.",
"Is this the last document?"
]

#### Create a bag-of-words representation of these documents

In [6]:
# create a CountVectorizer instance
vectorizer = CountVectorizer()

# transform the docs into vectors of word frequency
X = vectorizer.fit_transform(docs)

# convert the dense matrix to a data frame
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())

# print the data frame
df_tf

,but,document,first,is,it,last,neither,not,or,second,the,third,this
0,0,1,1,1,0,0,0,0,0,0,1,0,1
1,1,2,1,2,0,0,0,1,0,1,2,0,2
2,0,2,1,2,1,0,1,0,1,1,3,1,1
3,0,1,0,1,0,1,0,0,0,0,1,0,1


In [5]:
# examine the mapping of words to feature indexes
vectorizer.vocabulary_

{'this': 12,
 'is': 3,
 'the': 10,
 'first': 2,
 'document': 1,
 'second': 9,
 'but': 0,
 'not': 7,
 'third': 11,
 'it': 4,
 'neither': 6,
 'or': 8,
 'last': 5}

In [7]:
# X by default is a sparse matrix
# the todense() method will convert X to a dense matrix
X.todense()

matrix([[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
        [1, 2, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 2],
        [0, 2, 1, 2, 1, 0, 1, 0, 1, 1, 3, 1, 1],
        [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]], dtype=int64)

In [8]:
# examine the parameters used by the CountVectorizer instance
vectorizer

CountVectorizer()

For detailed usage, see http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

#### Use term presence instead of term frequency

In [9]:
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,but,document,first,is,it,last,neither,not,or,second,the,third,this
0,0,1,1,1,0,0,0,0,0,0,1,0,1
1,1,1,1,1,0,0,0,1,0,1,1,0,1
2,0,1,1,1,1,0,1,0,1,1,1,1,1
3,0,1,0,1,0,1,0,0,0,0,1,0,1


<b>TF-IDF</b><p>In Scikit-Learn, the actual formula used for tf-idf is tf * (idf + 1) = tf + tf * idf, instead of tf * idf. The effect of this is that terms with zero idf, i.e. that occur in all documents of a training set, will not be entirely ignored.</p>
<p>L1 normalization: TF is normalized using the sum of all TFs in a document</p>
<p>L2 normalization: TF is normalized using the root of the squared sum of all TFs in a document</p>

In [26]:
vectorizer = TfidfVectorizer(norm="l1")
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,but,document,first,is,it,last,neither,not,or,second,the,third,this
0,0.000000,0.191456,0.234178,0.191456,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.191456,0.000000,0.191456
1,0.131554,0.137301,0.083969,0.137301,0.000000,0.000000,0.000000,0.131554,0.000000,0.103719,0.137301,0.000000,0.137301
2,0.000000,0.108701,0.066478,0.108701,0.104151,0.000000,0.104151,0.000000,0.104151,0.082114,0.163051,0.104151,0.054350
3,0.000000,0.169025,0.000000,0.169025,0.000000,0.323901,0.000000,0.000000,0.000000,0.000000,0.169025,0.000000,0.169025


In [10]:
df_tf.apply(np.sum, axis=1)

0    1.0
1    1.0
2    1.0
3    1.0
dtype: float64

The sum of TFs for each document is 1 under L1 normalization

In [25]:
vectorizer = TfidfVectorizer(norm=None)
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,but,document,first,is,it,last,neither,not,or,second,the,third,this
0,0.000000,1.0,1.223144,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.0
1,1.916291,2.0,1.223144,2.0,0.000000,0.000000,0.000000,1.916291,0.000000,1.510826,2.0,0.000000,2.0
2,0.000000,2.0,1.223144,2.0,1.916291,0.000000,1.916291,0.000000,1.916291,1.510826,3.0,1.916291,1.0
3,0.000000,1.0,0.000000,1.0,0.000000,1.916291,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.0


#### Including bigrams and trigrams

In [27]:
docs = [
"This is the first document.", # "This is", "is the", "the first", "first document", "This is the","is the first", "the first document"
"This is the second document. But this is not the first document.",
"This is the third document. It is neither the first or the second document.",
"Is this the last document?"
]

In [28]:
vectorizer = CountVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,but,but this,but this is,document,document but,document but this,document it,document it is,first,first document,...,the third document,third,third document,third document it,this,this is,this is not,this is the,this the,this the last
0,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,1,0,1,0,0
1,1,1,1,2,1,1,0,0,1,1,...,0,0,0,0,2,2,1,1,0,0
2,0,0,0,2,0,0,1,1,1,0,...,1,1,1,1,1,1,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1


Now, there are 59 features.

In [29]:
vectorizer = CountVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,but this,document but,document it,first document,first or,is neither,is not,is the,is this,it is,...,not the,or the,second document,the first,the last,the second,the third,third document,this is,this the
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,1,0,1,0,0,1,1,0,0,...,1,0,1,1,0,1,0,0,2,0
2,0,0,1,0,1,1,0,1,0,1,...,0,1,1,1,0,1,1,1,1,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


#### Removing stopwords

### Removing stop words

In [15]:
vectorizer = CountVectorizer(ngram_range=(1,3), stop_words="english")
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,document,document document,document second,document second document,second,second document,second document document
0,1,0,0,0,0,0,0
1,2,1,0,0,1,1,1
2,2,0,1,1,1,1,0
3,1,0,0,0,0,0,0


In [16]:
vectorizer = CountVectorizer(ngram_range=(2,2), stop_words="english")
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,document document,document second,second document
0,0,0,0
1,1,0,1
2,0,1,1
3,0,0,0


In [17]:
word_list=['is','but']
vectorizer = CountVectorizer(ngram_range=(2,2), stop_words=word_list)
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,document it,document this,first document,first or,it neither,last document,neither the,not the,or the,second document,the first,the last,the second,the third,third document,this not,this the
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,0,1,1,0,0,0,0,1,0,1,1,0,1,0,0,1,1
2,1,0,0,1,1,0,1,0,1,1,1,0,1,1,1,0,1
3,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1


The number of features has reduced greatly.

#### Using a fixed vocabulary

In [18]:
vocab = ["first", "second", "third", "fourth"]
vectorizer = CountVectorizer(vocabulary=vocab)
X = vectorizer.fit_transform(docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,first,second,third,fourth
0,1,0,0,0
1,1,1,0,0
2,1,1,1,0
3,0,0,0,0


POS Tagging

In [30]:
# download nltk data first
nltk.download()
"""
After executing this cell, a window will pop up (NLTK Downloader).
You can select to downloand all packages.
This will take a few minutes.
Then, close the window.
"""

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


'\nAfter executing this cell, a window will pop up (NLTK Downloader).\nYou can select to downloand all packages.\nThis will take a few minutes.\nThen, close the window.\n'

In [31]:
stopwords = set(nltk.corpus.stopwords.words("english"))
stopwords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [20]:
# convert words to lower case
lower = map(str.lower, docs)

# replace punctuation with space
no_punc = map(lambda x: re.sub("[^a-z]", " ", x), lower)

# tokenize each document
tokenized = map(nltk.word_tokenize, no_punc)

# pos tag teach document
tagged = map(nltk.pos_tag, tokenized)

# remove stopwords
# stopwords can only be removed after POS tags are generated. Otherwise, it will influence the POS tagging results.
stopwords = nltk.corpus.stopwords.words("english")
def remove_stopwords(doc):
    out = []
    for word in doc:
        if word[0] not in stopwords: out.append(word)
    return out

no_stopwords = list(map(remove_stopwords, tagged))

no_stopwords

[[('first', 'JJ'), ('document', 'NN')],
 [('second', 'JJ'), ('document', 'NN'), ('first', 'JJ'), ('document', 'NN')],
 [('third', 'JJ'),
  ('document', 'NN'),
  ('neither', 'CC'),
  ('first', 'JJ'),
  ('second', 'JJ'),
  ('document', 'NN')],
 [('last', 'JJ'), ('document', 'NN')]]

In [32]:
docs = [
"This is the first document.",
"This is the second document. But this is not the first document.",
"This is the third document. It is neither the first or the second document.",
"Is this the last document?"
]

Tag description: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [33]:
# convert the lists of tagged words to string so that scikit-learn can tokenize them
tagged_docs = list(map(str, no_stopwords))
tagged_docs

["[('first', 'JJ'), ('document', 'NN')]",
 "[('second', 'JJ'), ('document', 'NN'), ('first', 'JJ'), ('document', 'NN')]",
 "[('third', 'JJ'), ('document', 'NN'), ('neither', 'CC'), ('first', 'JJ'), ('second', 'JJ'), ('document', 'NN')]",
 "[('last', 'JJ'), ('document', 'NN')]"]

In [24]:
# vecotrize
vectorizer = CountVectorizer(token_pattern=r"\('[^ ]+', '[^ ]+'\)", lowercase=False)
X = vectorizer.fit_transform(tagged_docs)
df_tf = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df_tf

,"('document', 'NN')","('first', 'JJ')","('last', 'JJ')","('neither', 'CC')","('second', 'JJ')","('third', 'JJ')"
0,1,1,0,0,0,0
1,2,1,0,0,1,0
2,2,1,0,1,1,1
3,1,0,1,0,0,0
